In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 119 kB in 2s (60.2 kB/s)
Reading package lists... Done


In [27]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=c74dabea95e01267ef8e515856c6a1d76a7f5d09f47cb4dc3948736ef06a2aad
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
home_sales_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView('home_sales')


In [7]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("SELECT ROUND(AVG(price), 2) as average_price, YEAR(date) as year_sale, bedrooms FROM home_sales WHERE bedrooms ='4' GROUP BY YEAR(date), bedrooms ORDER BY YEAR(date)").show()


+-------------+---------+--------+
|average_price|year_sale|bedrooms|
+-------------+---------+--------+
|     300263.7|     2019|       4|
|    298353.78|     2020|       4|
|    301819.44|     2021|       4|
|    296363.88|     2022|       4|
+-------------+---------+--------+



In [18]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("SELECT ROUND(AVG(price), 2) as average_price, date_built, bedrooms,bathrooms FROM home_sales WHERE bedrooms ='3' and bathrooms='3' GROUP BY date_built, bedrooms, bathrooms ORDER BY date_built").show()


+-------------+----------+--------+---------+
|average_price|date_built|bedrooms|bathrooms|
+-------------+----------+--------+---------+
|    292859.62|      2010|       3|        3|
|    291117.47|      2011|       3|        3|
|    293683.19|      2012|       3|        3|
|    295962.27|      2013|       3|        3|
|    290852.27|      2014|       3|        3|
|     288770.3|      2015|       3|        3|
|    290555.07|      2016|       3|        3|
|    292676.79|      2017|       3|        3|
+-------------+----------+--------+---------+



In [12]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("""
SELECT ROUND(AVG(price), 2) as average_price, date_built,
FROM home_sales
WHERE bedrooms ='3' AND bathrooms='3' AND floors='2' AND sqft_living >='2000'
GROUP BY date_built
ORDER BY date_built
""").show()



+-------------+----------+
|average_price|date_built|
+-------------+----------+
|    285010.22|      2010|
|    276553.81|      2011|
|    307539.97|      2012|
|    303676.79|      2013|
|    298264.72|      2014|
|    297609.97|      2015|
|     293965.1|      2016|
|    280317.58|      2017|
+-------------+----------+



In [14]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
SELECT ROUND(AVG(price), 2) as average_price, view
FROM home_sales
WHERE price >='350000'
GROUP BY view
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+-------------+----+
|average_price|view|
+-------------+----+
|    403005.77|   7|
|    788128.21|  51|
|     404673.3|  15|
|    798684.82|  54|
|    399548.12|  11|
|    397771.65|  29|
|    750537.94|  69|
|     396964.5|  42|
|    719078.95|  87|
|    752861.18|  73|
|    767036.67|  64|
|     398867.6|   3|
|     397862.0|  30|
|    401419.75|  34|
|     791453.0|  59|
|    398592.71|   8|
|    402124.62|  28|
|    402022.68|  22|
|     742853.8|  85|
|    399586.53|  16|
+-------------+----+
only showing top 20 rows

--- 0.849651575088501 seconds ---


In [15]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [16]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [25]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""
SELECT ROUND(AVG(price), 2) as average_price, view
FROM home_sales
WHERE price >='350000'
GROUP BY view
""").show()

print("--- %s seconds ---" % (time.time() - start_time))


+-------------+----+
|average_price|view|
+-------------+----+
|    403005.77|   7|
|    788128.21|  51|
|     404673.3|  15|
|    798684.82|  54|
|    399548.12|  11|
|    397771.65|  29|
|    750537.94|  69|
|     396964.5|  42|
|    719078.95|  87|
|    752861.18|  73|
|    767036.67|  64|
|     398867.6|   3|
|     397862.0|  30|
|    401419.75|  34|
|     791453.0|  59|
|    398592.71|   8|
|    402124.62|  28|
|    402022.68|  22|
|     742853.8|  85|
|    399586.53|  16|
+-------------+----+
only showing top 20 rows

--- 0.38677263259887695 seconds ---


In [18]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

In [19]:
# 11. Read the parquet formatted data.
home_sales_df_p=spark.read.parquet('home_sales_partitioned')

In [20]:
# 12. Create a temporary table for the parquet data.
home_sales_df_p.createOrReplaceTempView('home_sales_p')

In [27]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""
SELECT ROUND(AVG(price), 2) as average_price, view
FROM home_sales_p
WHERE price >='350000'
GROUP BY view
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+-------------+----+
|average_price|view|
+-------------+----+
|    403005.77|   7|
|    788128.21|  51|
|     404673.3|  15|
|    798684.82|  54|
|    399548.12|  11|
|    397771.65|  29|
|    750537.94|  69|
|     396964.5|  42|
|    752861.18|  73|
|    719078.95|  87|
|    767036.67|  64|
|     398867.6|   3|
|     397862.0|  30|
|    401419.75|  34|
|     791453.0|  59|
|    398592.71|   8|
|    402124.62|  28|
|    402022.68|  22|
|     742853.8|  85|
|    401934.21|  35|
+-------------+----+
only showing top 20 rows

--- 0.4212520122528076 seconds ---


In [28]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [29]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False